
# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>

Le projet a été développé à l'aide de Alice Breton, étudiante à la maîtrise en génie informatique. Elle a suivi le cours lors de la session Hiver 2019.



# Projet : Débordement d'égouts

La description du projet est disponible à l'adresse suivante :
https://www.kaggle.com/t/a238b752c33a41d9803c2cdde6bfc929

Ce calepin Jupyter de base permet de charger et de nettoyer les données fournies. La dernière section détaille la génération du fichier des prédictions afin de le soumettre sur Kaggle dans le bon format.

Dans un premier temps, vous devrez récupérer l'archive *data.zip* sur Moodle. Ce dossier contient les fichiers suivants :
- surverses.csv
- precipitation.csv
- ouvrages-surverses.csv
- test.csv

Veuillez le décompresser dans le répertoire de ce calepin.

Le fichier *surverse.csv* répertorie s'il y a surverse (1) ou non (0) au cours de la journée pour les 170 ouvrages de débordement de 2013 à 2018 pour les mois de mai à octobre (inclusivement). Des renseignements additionnels sur les données sont disponibles à l'adresse suivante :

http://donnees.ville.montreal.qc.ca/dataset/debordement


Le fichier *precipitation.csv* contient les précipitations horaires en dixième de *mm* enregistrées à 5 stations pluviométriques de 2013 à 2019 :
- McTavish (7024745)
- Ste-Anne-de-Bellevue (702FHL8)
- Montreal/Pierre Elliott Trudeau Intl (702S006)
- Montreal/St-Hubert (7027329)
- L’Assomption (7014160)

Plus d'informations sur les précipitations sont disponibles à l'adresse suivante :

https://climat.meteo.gc.ca/climate_data/hourly_data_f.html?hlyRange=2008-01-08%7C2019-11-12&dlyRange=2002-12-23%7C2019-11-12&mlyRange=%7C&StationID=30165&Prov=QC&urlExtension=_f.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2019&selRowPerPage=25&Line=17&searchMethod=contains&Month=11&Day=12&txtStationName=montreal&timeframe=1&Year=2019

Le fichier *ouvrages-surverses.csv* contient différentes caractéristiques des ouvrages de débordement. 

http://donnees.ville.montreal.qc.ca/dataset/ouvrage-surverse

Le fichier *test.csv* contient les ouvrages et les jours pour lesquels vous devez prédire s'il y a eu surverse (true) ou non (false). Notez que l'on s'intéresse ici à 5 ouvrages de débordement localisés tout autour de l'Ile de Montréal :
- 3260-01D dans Rivière-des-Prairies 
- 3350-07D dans Ahunstic 
- 4240-01D dans Pointe-aux-Trembles 
- 4350-01D dans le Vieux-Montréal 
- 4380-01D dans Verdun

#### Remarque

Dans le projet, on ne s'intéresse qu'aux surverses occasionnées par les précipitations. On ignore les surverses occasionnées par 
- fonte de neige (F)
- travaux planifiés et entretien (TPL)
- urgence (U)
- autre (AUT)

On suppose que lorsqu'il n'y a pas de raison pour la surverse, il s'agit d'une surverse causée par les précipitations. Puisque Nous nous intéresserons uniquement aux surverses occasionnées par les précipitations liquides, nous ne considérons que les mois de mai à octobre inclusivement.

In [35]:
using ScikitLearn
@sk_import naive_bayes: GaussianNB

PyObject <class 'sklearn.naive_bayes.GaussianNB'>

In [33]:
# import Pkg; Pkg.add("Conda")
import Conda
Conda.add("scikit-learn")

 Resolving package versions...
  Updating `C:\Users\Mouns\.julia\environments\v1.2\Project.toml`
 [no changes]
  Updating `C:\Users\Mouns\.julia\environments\v1.2\Manifest.toml`
 [no changes]


┌ Info: Running `conda install -y scikit-learn` in root environment
└ @ Conda C:\Users\Mouns\.julia\packages\Conda\kLXeC\src\Conda.jl:112


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Mouns\.julia\conda\3

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.27 |                0         163 KB
    certifi-2019.11.28         |           py36_0         157 KB
    joblib-0.14.0              |             py_0         201 KB
    scikit-learn-0.21.3        |   py36h6288b17_0         4.4 MB
    scipy-1.3.1                |   py36h29ff71c_0        11.3 MB
    ------------------------------------------------------------
                                           Total:        16.2 MB

The following NEW packages will be INSTALLED:

  joblib             pkgs/main/noarch::joblib-0.14.0-py_0
  scikit-learn       pkgs/main/win-64::scikit-learn-0.21.3-py36h6288b17_0
  scipy              pkgs/main/win-64::scipy-1.3.1-py36h29f

In [14]:
using CSV, DataFrames, Statistics, Dates, Gadfly, Random, LinearAlgebra, GLM, Distributions
include("functions.jl");

# Chargement des données et nettoyage préliminaire

## Chargement et nettoyage des surverses et précipitations

In [15]:
data = CSV.read("data/surverses.csv",missingstring="-99999")
first(data,5)

# Extraction des surverses pour les mois de mai à octobre inclusivement
data = filter(row -> month(row.DATE) > 4, data) 
data = filter(row -> month(row.DATE) < 11, data) 

# Remplacement des valeurs *missing* dans la colonne :RAISON par "Inconnue"
raison = coalesce.(data[:,:RAISON],"Inconnue")
data[!,:RAISON] = raison

# Exlusion des surverses coccasionnées par d'autres facteurs que les précipitations liquides
data = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], data) 
select!(data, [:NO_OUVRAGE, :DATE, :SURVERSE])

surverse_df = dropmissing(data, disallowmissing=true)

### CHARGEMENT DES PRÉCIPITATIONS
data = CSV.read("data/precipitations.csv",missingstring="-99999")
rename!(data, Symbol("St-Hubert")=>:StHubert)

# Nettoyage des données sur les précipitations
data = filter(row -> month(row.date) > 4, data) 
data = filter(row -> month(row.date) < 11, data) 


,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-05-01,0,0,0,0,0,missing
2,2013-05-01,1,0,0,0,0,missing
3,2013-05-01,2,0,0,0,0,missing
4,2013-05-01,3,0,0,0,0,missing
5,2013-05-01,4,0,0,0,0,missing
6,2013-05-01,5,0,0,0,0,missing
7,2013-05-01,6,0,0,0,0,missing
8,2013-05-01,7,0,0,0,0,missing
9,2013-05-01,8,0,0,0,0,missing


In [16]:
# REMPLACER LES VALEURS MANQUANTES DANS LE DATAFRAME DE PRÉCIPITATIONS
# On veut remplacer les données manquante grâce à une simple régression linéaire (Ridge).
# Pour ce faire, chaque station météo agira comme variable explicative pour les 4 autres
# On drop les rangées dont les 5valeurs sont missing

data_full = deepcopy(data)
dropmissing!(data, disallowmissing=true)

models = Dict()

for i = 1:length(data_full[:, 1])
    if any(ismissing, data_full[i, 3:7]) #ismissing c est pas une variable mais une méthode de Julia, 3:7 pour les 5 stations
        missing = []
        notmissing = []
        for j=3:7 #on passe à travers les 5 colonnes pour regarder celles qui sont missing ou non
            if ismissing(data_full[i, j])
                push!(missing, names(data_full)[j]) #push généralisé à tous les conteneurs
                #Le point d exclam ation pour dire à la fonction de modifier le conteneur qu on lui passe
            else
                push!(notmissing, names(data_full)[j])
            end
        end
        
        if length(notmissing) > 0 #si on a 5 missing sur la ligne on peut juste rien prédire donc on skip ça
            for m in missing #m c est la station à prédire
                if !haskey(models, (m, notmissing)) #ça véfirie si dans le dictionnaire models il y a la key (m, notmissing)
                    models[(m, notmissing)] = CoeffRidge(data, m, notmissing) #on change la case dans le dico palr le coef que la fonction nous retourne
                end
                data_full[i, m] = round(convert(Vector{Float64}, data_full[i, notmissing])' * models[(m, notmissing)]) # on ajoute à la case manque la nouvelle valeur = x*Beta
            end
        end
    end
end

dropmissing!(data_full)
data_full

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64,Int64
1,2013-05-01,0,0,0,0,0,0
2,2013-05-01,1,0,0,0,0,0
3,2013-05-01,2,0,0,0,0,0
4,2013-05-01,3,0,0,0,0,0
5,2013-05-01,4,0,0,0,0,0
6,2013-05-01,5,0,0,0,0,0
7,2013-05-01,6,0,0,0,0,0
8,2013-05-01,7,0,0,0,0,0
9,2013-05-01,8,0,0,0,0,0


## Extraction variables explicatives (Maude L.)

In [17]:
# Maximum de précipitations en 1 heure dans une journée.
X_pcp_max = by(data_full, :date,  McTavish = :McTavish=>maximum, Bellevue = :Bellevue=>maximum, 
   Assomption = :Assomption=>maximum, Trudeau = :Trudeau=>maximum, StHubert = :StHubert=>maximum)

# Somme des précipitations dans une journée
X_pcp_sum = by(data_full, :date,  McTavish = :McTavish=>sum, Bellevue = :Bellevue=>sum, 
   Assomption = :Assomption=>sum, Trudeau = :Trudeau=>sum, StHubert = :StHubert=>sum)

# Précipitation à chaque heure pour chaque journée
X_pcp_hour = data_full

# Somme des précipitations des 2 heures précédentes

X_pcp_three_hours = copy(data_full)
buffer = copy(data_full)
for i=3:7
    for j=1:length(data_full[:,2])
        if (j > 2)
            X_pcp_three_hours[j, i] = (buffer[(j-2), i] + buffer[(j-1), i] + buffer[j,i])
        else
            X_pcp_three_hours[j, i] = buffer[j, i]
        end
    end
end

# Somme des précipitations des 2 jours précédents

X_pcp_two_days = copy(X_pcp_sum)
buffer = copy(X_pcp_sum)
for i=2:6
    for j=1:length(X_pcp_sum[:,2])
        if (j > 1)
            X_pcp_two_days[j, i] = buffer[(j-1), i] + buffer[j,i]
        else
            X_pcp_two_days[j, i] = buffer[j, i]
        end
    end
end


# Mois de la prédiction pour chaque ouvrage
X_date = DataFrame(NO_OUVRAGE=surverse_df[:,:NO_OUVRAGE], MOIS=zeros(Int64,length(surverse_df[:,1])), SURVERSE=surverse_df[:,:SURVERSE])
for i=1:length(surverse_df[:,1])
    X_date[i,:MOIS] = month(surverse_df[i,:DATE])
end

X_mois = DataFrame(MOIS=["Mai", "Juin", "Juillet", "Août", "Septembre", "Octobre"], RivierePrairie=zeros(Int64,6), Ahuntsic=zeros(Int64,6), PointeAuxTrembles=zeros(Int64,6), VieuxMontreal=zeros(Int64,6), Verdun=zeros(Int64,6))
for j=1:length(X_date[:,1])
    if(X_date[j,1] == "3260-01D")
        X_mois[(X_date[j,2]-4),:RivierePrairie] += (X_date[j,3])
    elseif (X_date[j,1] == "3350-07D")
        X_mois[(X_date[j,2]-4),:Ahuntsic] += (X_date[j,3])
    elseif (X_date[j,1] == "4240-01D")
        X_mois[(X_date[j,2]-4),:PointeAuxTrembles] += (X_date[j,3])
    elseif (X_date[j,1] == "4350-01D")
        X_mois[(X_date[j,2]-4),:VieuxMontreal] += (X_date[j,3])
    elseif (X_date[j,1] == "4380-01D")
        X_mois[(X_date[j,2]-4),:Verdun] += (X_date[j,3])
    end
end


In [18]:
# Création de 5 dataframes séparés pour les données de surverses des 5 ouvrages d'intérêt
ouvrage = ["3260-01D", "3350-07D", "4240-01D", "4350-01D", "4380-01D"]

filtered_surverse = filter(row -> row.NO_OUVRAGE == ouvrage[1], surverse_df)
surverse_rivierePrairie = DataFrame(DATE=filtered_surverse[:, 2], RIVIEREPRAIRIE=filtered_surverse[:, 3])

filtered_surverse = filter(row -> row.NO_OUVRAGE == ouvrage[2], surverse_df)
surverse_ahuntsic = DataFrame(DATE=filtered_surverse[:, 2], AHUNTSIC=filtered_surverse[:, 3])

filtered_surverse = filter(row -> row.NO_OUVRAGE == ouvrage[3], surverse_df)
surverse_pointeAuxTrembles = DataFrame(DATE=filtered_surverse[:, 2], POINTEAUXTREMBLES=filtered_surverse[:, 3])

filtered_surverse = filter(row -> row.NO_OUVRAGE == ouvrage[4], surverse_df)
surverse_vieuxMontreal = DataFrame(DATE=filtered_surverse[:, 2], VIEUXMONTREAL=filtered_surverse[:, 3])

filtered_surverse = filter(row -> row.NO_OUVRAGE == ouvrage[5], surverse_df)
surverse_verdun = DataFrame(DATE=filtered_surverse[:, 2], VERDUN=filtered_surverse[:, 3]);


# Analyse exploratoire

Cette section consitue une analyse exploratoire superficielle permettant de voir s'il existe un lien entre les précipitations et les surverses.

Prenons arbitrairement l'ouvrage de débordement près du Bota-Bota (4350-01D). La station météorologique la plus proche est McTavish. Prenons deux variables explicatives simple :
- la somme journalière des précipitations
- le taux horaire maximum journalier de précipitations

#### Calcul de la quantité journalière de précipitations pour chacune des stations météorologiques

In [19]:
df = DataFrame(SURVERSES = surverse_rivierePrairie[:, 2], SOMME = X_pcp_sum[]  )
plot(df, x=:SURVERSE, y=:SUM, Geom.boxplot)

MethodError: MethodError: no method matching getindex(::DataFrame)
Closest candidates are:
  getindex(::DataFrame, !Matched::Colon) at deprecated.jl:65
  getindex(::DataFrame, !Matched::Colon, !Matched::Union{Signed, Symbol, Unsigned}) at C:\Users\Mouns\.julia\packages\DataFrames\yH0f6\src\dataframe\dataframe.jl:352
  getindex(::DataFrame, !Matched::Colon, !Matched::Union{Colon, Regex, AbstractArray{T,1} where T, All, Between, InvertedIndex}) at C:\Users\Mouns\.julia\packages\DataFrames\yH0f6\src\dataframe\dataframe.jl:401
  ...

# Régression logistique à 5 variables explicatives pour l'ouvrage Vieux-Montréal

In [20]:
# Construction d'un dataframe contenant toutes les variables explicatives
#ouvrage = "3260-01D"

function TrainLogit(ouvrage, surverse_df)
    df_train = filter(row -> row.NO_OUVRAGE == ouvrage, surverse_df)

    x₁ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière McTavish
    x₂ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière Bellevue
    x₃ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière Assomption
    x₄ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière St-Hubert
    x₅ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière PET
    x₆ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours McTavish
    x₇ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours Bellevue
    x₈ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours Assomption
    x₉ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours St-Hubert
    x₁₀ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours PET
    x₁₁ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier McTavish
    x₁₂ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier Bellevue
    x₁₃ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier Assomption
    x₁₄ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier St-Hubert
    x₁₅ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier PET



    for i=1:size(df_train,1)

        ind = findfirst(X_pcp_sum[:,:date] .== df_train[i,:DATE])

        x₁[i] = X_pcp_sum[ind,:McTavish]

        x₂[i] = X_pcp_sum[ind,:Bellevue]

        x₃[i] = X_pcp_sum[ind,:Assomption]

        x₄[i] = X_pcp_sum[ind,:StHubert]

        x₅[i] = X_pcp_sum[ind,:Trudeau]

        x₆[i] = X_pcp_two_days[ind, :McTavish]

        x₇[i] = X_pcp_two_days[ind, :Bellevue]

        x₈[i] = X_pcp_two_days[ind, :Assomption]

        x₉[i] = X_pcp_two_days[ind, :StHubert]

        x₁₀[i] = X_pcp_two_days[ind, :Trudeau]

        x₁₁[i] = X_pcp_max[ind, :McTavish]

        x₁₂[i] = X_pcp_max[ind, :Bellevue]

        x₁₃[i] = X_pcp_max[ind, :Assomption]

        x₁₄[i] = X_pcp_max[ind, :StHubert]

        x₁₅[i] = X_pcp_max[ind, :Trudeau]

    end

    df_train[!,:SUMMcTavish] = x₁
    df_train[!,:SUMBellevue] = x₂
    df_train[!,:SUMAssomption] = x₃
    df_train[!,:SUMStHubert] = x₄
    df_train[!,:SUMPET] = x₅
    df_train[!,:SUM2McTavish] = x₆
    df_train[!,:SUM2Bellevue] = x₇
    df_train[!,:SUM2Assomption] = x₈
    df_train[!,:SUM2StHubert] = x₉
    df_train[!,:SUM2PET] = x₁₀
    df_train[!,:MAXMcTavish] = x₁₁
    df_train[!,:MAXBellevue] = x₁₂
    df_train[!,:MAXAssomption] = x₁₃
    df_train[!,:MAXStHubert] = x₁₄
    df_train[!,:MAXPET] = x₁₅

    M = glm(@formula(SURVERSE ~ 1 + SUMMcTavish + SUMBellevue + SUMAssomption + SUMStHubert + SUMPET + SUM2McTavish + SUM2Bellevue + SUM2Assomption + SUM2StHubert + SUM2PET + MAXMcTavish + MAXBellevue + MAXAssomption + MAXStHubert + MAXPET), df_train,  Bernoulli(), LogitLink())
    
    return M

end


TrainLogit (generic function with 1 method)

In [21]:
# f1 = computeF1score(Ŷ, test_set[:, :SURVERSE])
# print(f1)

In [22]:
# #utilisation du score F1 pour calculer le meilleur seuil
# seuils = LinRange(0, 1, 200)
# meilleur_seuil = 0
# meilleur_f1 = 0

# for i = 1:length(seuils)
#     Ŷ = zeros(Int64,length(test_set[:,1]))
#     Ŷ[θ̂.>seuils[i]] .= 1;
#     f1 = computeF1score(Ŷ, test_set[:, :SURVERSE])
#     if f1 > meilleur_f1
#         meilleur_f1 = f1
#         meilleur_seuil = seuils[i]
#     end
# end

# [meilleur_seuil meilleur_f1]

## Bayesien Naïf (Othman et Anis)

In [23]:
train_df, test_df = splitdataframe(surverse_df, 0.7)


(112769×3 DataFrame
│ Row    │ NO_OUVRAGE │ DATE       │ SURVERSE │
│        │ String     │ Date       │ Int64    │
├────────┼────────────┼────────────┼──────────┤
│ 1      │ 0642-01D   │ 2013-05-01 │ 0        │
│ 2      │ 0642-01D   │ 2013-05-02 │ 0        │
│ 3      │ 0642-01D   │ 2013-05-05 │ 0        │
│ 4      │ 0642-01D   │ 2013-05-07 │ 0        │
│ 5      │ 0642-01D   │ 2013-05-08 │ 0        │
│ 6      │ 0642-01D   │ 2013-05-09 │ 0        │
│ 7      │ 0642-01D   │ 2013-05-10 │ 0        │
│ 8      │ 0642-01D   │ 2013-05-12 │ 0        │
│ 9      │ 0642-01D   │ 2013-05-13 │ 0        │
│ 10     │ 0642-01D   │ 2013-05-14 │ 0        │
⋮
│ 112759 │ 4795-01D   │ 2018-10-18 │ 0        │
│ 112760 │ 4795-01D   │ 2018-10-21 │ 0        │
│ 112761 │ 4795-01D   │ 2018-10-22 │ 0        │
│ 112762 │ 4795-01D   │ 2018-10-23 │ 0        │
│ 112763 │ 4795-01D   │ 2018-10-24 │ 0        │
│ 112764 │ 4795-01D   │ 2018-10-25 │ 0        │
│ 112765 │ 4795-01D   │ 2018-10-26 │ 0        │
│ 112766 │ 4795-01

In [36]:
?GaussianNB


search: GaussianNB Gaussian InverseGaussian NormalInverseGaussian




    Gaussian Naive Bayes (GaussianNB)

    Can perform online updates to model parameters via `partial_fit` method.
    For details on algorithm used to update feature means and variance online,
    see Stanford CS tech report STAN-CS-79-773 by Chan, Golub, and LeVeque:

        http://i.stanford.edu/pub/cstr/reports/cs/tr/79/773/CS-TR-79-773.pdf

    Read more in the :ref:`User Guide <gaussian_naive_bayes>`.

    Parameters
    ----------
    priors : array-like, shape (n_classes,)
        Prior probabilities of the classes. If specified the priors are not
        adjusted according to the data.

    var_smoothing : float, optional (default=1e-9)
        Portion of the largest variance of all features that is added to
        variances for calculation stability.

    Attributes
    ----------
    class_prior_ : array, shape (n_classes,)
        probability of each class.

    class_count_ : array, shape (n_classes,)
        number of training samples observed in each class.

    thet

In [28]:
# Construction d'un dataframe contenant toutes les variables explicatives
#ouvrage = "3260-01D"

function get_var_exp(ouvrage, surverse_df)
    df_train = filter(row -> row.NO_OUVRAGE == ouvrage, surverse_df)

    x₁ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière McTavish
    x₂ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière Bellevue
    x₃ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière Assomption
    x₄ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière St-Hubert
    x₅ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière PET
    x₆ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours McTavish
    x₇ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours Bellevue
    x₈ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours Assomption
    x₉ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours St-Hubert
    x₁₀ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme des 2 jours PET
    x₁₁ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier McTavish
    x₁₂ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier Bellevue
    x₁₃ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier Assomption
    x₁₄ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier St-Hubert
    x₁₅ = Array{Int64}(undef, size(df_train,1)) # variable pour le max journalier PET



    for i=1:size(df_train,1)

        ind = findfirst(X_pcp_sum[:,:date] .== df_train[i,:DATE])

        x₁[i] = X_pcp_sum[ind,:McTavish]

        x₂[i] = X_pcp_sum[ind,:Bellevue]

        x₃[i] = X_pcp_sum[ind,:Assomption]

        x₄[i] = X_pcp_sum[ind,:StHubert]

        x₅[i] = X_pcp_sum[ind,:Trudeau]

        x₆[i] = X_pcp_two_days[ind, :McTavish]

        x₇[i] = X_pcp_two_days[ind, :Bellevue]

        x₈[i] = X_pcp_two_days[ind, :Assomption]

        x₉[i] = X_pcp_two_days[ind, :StHubert]

        x₁₀[i] = X_pcp_two_days[ind, :Trudeau]

        x₁₁[i] = X_pcp_max[ind, :McTavish]

        x₁₂[i] = X_pcp_max[ind, :Bellevue]

        x₁₃[i] = X_pcp_max[ind, :Assomption]

        x₁₄[i] = X_pcp_max[ind, :StHubert]

        x₁₅[i] = X_pcp_max[ind, :Trudeau]

    end

    df_train[!,:SUMMcTavish] = x₁
    df_train[!,:SUMBellevue] = x₂
    df_train[!,:SUMAssomption] = x₃
    df_train[!,:SUMStHubert] = x₄
    df_train[!,:SUMPET] = x₅
    df_train[!,:SUM2McTavish] = x₆
    df_train[!,:SUM2Bellevue] = x₇
    df_train[!,:SUM2Assomption] = x₈
    df_train[!,:SUM2StHubert] = x₉
    df_train[!,:SUM2PET] = x₁₀
    df_train[!,:MAXMcTavish] = x₁₁
    df_train[!,:MAXBellevue] = x₁₂
    df_train[!,:MAXAssomption] = x₁₃
    df_train[!,:MAXStHubert] = x₁₄
    df_train[!,:MAXPET] = x₁₅

    return df_train

end


get_var_exp (generic function with 1 method)

In [41]:
df_train = get_var_exp("3260-01D", train_df)
X_train = df_test[4:end]
y_train = df_test[3]
model = GaussianNB().fit(X_train, y_train)

┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at In[41]:2
└ @ Core In[41]:2
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[41]:3
└ @ Core In[41]:3


PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= C:\Users\Mouns\.julia\packages\PyCall\ttONZ\src\pyfncall.jl:44 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'ValueError'>
ValueError('Expected 2D array, got scalar array instead:\narray=<PyCall.jlwrap 757×15 DataFrame\n│ Row │ SUMMcTavish │ SUMBellevue │ SUMAssomption │ SUMStHubert │ SUMPET │ SUM2McTavish │ SUM2Bellevue │ SUM2Assomption │ SUM2StHubert │ SUM2PET │ MAXMcTavish │ MAXBellevue │ MAXAssomption │ MAXStHubert │ MAXPET │\n│     │ Int64       │ Int64       │ Int64         │ Int64       │ Int64  │ Int64        │ Int64        │ Int64          │ Int64        │ Int64   │ Int64       │ Int64       │ Int64         │ Int64       │ Int64  │\n├─────┼─────────────┼─────────────┼───────────────┼─────────────┼────────┼──────────────┼──────────────┼────────────────┼──────────────┼─────────┼─────────────┼─────────────┼───────────────┼─────────────┼────────┤\n│ 1   │ 0           │ 0           │ 0      

In [27]:
threshold = 200

#X1: Somme de precipitation max de McTavish
ouvrage = "3260-01D"
df_train = filter(row -> row.NO_OUVRAGE == ouvrage, train_df)
x₁ = Array{Int64}(undef, size(df_train,1)) # variable pour la somme journalière McTavish
# println(df_train)

for i=1:size(df_train,1)
    ind = findfirst(X_pcp_sum[:,:date] .== df_train[i,:DATE])
    x₁[i] = X_pcp_sum[ind,:McTavish]
end
df_train[!,:SUMMcTavish] = x₁

println(df_train[1:10,:])

# # On get les jours ou y a surverse et non
# surverse_days = filter(row -> row.SURVERSE == 1, df_train)
# non_surverse_days = filter(row -> row.SURVERSE == 0, df_train)

# n₀ = length(non_surverse_days[:SURVERSE])
# n₁ = length(surverse_days[:SURVERSE])
# n = n₁ + n₀

# # nombre d'entrées sans surverses ou le max de preci est grand
# rainy_no_surverse = filter(row -> row.SUMMcTavish >= threshold, non_surverse_days)
# n₀₁ = length(rainy_no_surverse[:SURVERSE])

# # nombre d'entrées avec surverses ou le max de preci est grand
# rainy_surverse = filter(row -> row.SUMMcTavish >= threshold, surverse_days)
# n₁₁ = length(rainy_surverse[:SURVERSE])

# # nombre d'entrées sans surverses ou le max de preci est petit
# non_rainy_no_surverse = filter(row -> row.SUMMcTavish < threshold, non_surverse_days)
# n₀₀ = length(rainy_no_surverse[:SURVERSE])

# # nombre d'entrées avec surverses ou le max de preci est peti
# non_rainy_surverse = filter(row -> row.SUMMcTavish < threshold, surverse_days)
# n₁₀ = length(rainy_surverse[:SURVERSE])

# q_surverse = (n₁+1)/(n+2) * (n₁₁+1)/(n₁+2)
# q_normal = (n₀+1)/(n+2) * (n₀₁+1)/(n₀+2)

# # constante de normalisation
# c = q_surverse + q_normal

# p_surverse = q_surverse/c

10×4 DataFrame
│ Row │ NO_OUVRAGE │ DATE       │ SURVERSE │ SUMMcTavish │
│     │ String     │ Date       │ Int64    │ Int64       │
├─────┼────────────┼────────────┼──────────┼─────────────┤
│ 1   │ 3260-01D   │ 2013-05-01 │ 0        │ 0           │
│ 2   │ 3260-01D   │ 2013-05-02 │ 0        │ 0           │
│ 3   │ 3260-01D   │ 2013-05-03 │ 0        │ 0           │
│ 4   │ 3260-01D   │ 2013-05-05 │ 0        │ 0           │
│ 5   │ 3260-01D   │ 2013-05-08 │ 0        │ 0           │
│ 6   │ 3260-01D   │ 2013-05-10 │ 0        │ 0           │
│ 7   │ 3260-01D   │ 2013-05-12 │ 0        │ 0           │
│ 8   │ 3260-01D   │ 2013-05-13 │ 0        │ 10          │
│ 9   │ 3260-01D   │ 2013-05-14 │ 0        │ 0           │
│ 10  │ 3260-01D   │ 2013-05-17 │ 0        │ 0           │


In [13]:
q_surverse_test = (n₁+1)/(n+2) * (n₁ - n₁₁+1)/(n₁+2)
q_normal_test = (n₀+1)/(n+2) * (n₀ - n₀₁+1)/(n₀+2)
 
# constante de normalisation
c = q_surverse_test + q_normal_test

p_surverse_test = q_surverse_test/c

0.04288558021414625

In [54]:
## X1: Max de precipitation sur la station McTavish 
surverse_days = filter(row -> row.SURVERSE == 1, train_df) 
normal_days = filter(row -> row.SURVERSE == 0, train_df)

# nombre total de surverses
n₀ = length(normal_days[:SURVERSE])
# nombre total de nom surverse
n₁ = length(surverse_days[:SURVERSE])

# nombre d'entrées dans l'ensemble d'entraînement
n = n₀ + n₁

# # nombre d'entrées sans surverses ou le max de preci est grand
jour_max = filter(row -> any( i -> (i > threshold), row[2:6]), X_pcp_max)[1]
normal_days_max = filter(row -> row.DATE in jour_max, normal_days)
n₀₁ = length(normal_days_max[:SURVERSE])
println(n₀₁)
# # nombre d'entrées sans surverses ou le max de preci est petit
n₀₀ = n₀ - n₀₁

# # nombre d'entrées avec surverses ou le max de preci est grand
surverse_days_max = filter(row -> row.DATE in jour_max, surverse_days)
n₁₁ = length(surverse_days_max[:SURVERSE])
println(n₁₁)
# # nombre d'entrées avec surverses ou le max de preci est petit
n₁₀ = n₁ - n₁₁

q_surverse = (n₁+1)/(n+2) * (n₁₁+1)/(n₁+2)
q_normal = (n₀+1)/(n+2) * (n₀₁+1)/(n₀+2)

# constante de normalisation
c = q_surverse + q_normal

p_surverse = q_surverse/c

# println("Si le mot http est présent dans le message, il y a une prob de $p_spam que ce soit un spam.")

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[54]:4
└ @ Core In[54]:4
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[54]:7
└ @ Core In[54]:7


2413
719


┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[54]:12
└ @ Core In[54]:12
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[54]:16
└ @ Core In[54]:16
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[54]:23
└ @ Core In[54]:23


0.22969489142673852

# Création du fichier de prédictions pour soumettre sur Kaggle

Dans ce cas-ci, nous prédirons une surverse avec une probabilité de 1/2 sans considérer aucune variable explicative.

In [7]:
## Chargement du fichier de test
test = CSV.read("data/test.csv")
hcat(test, DataFrame(SURVERSE=zeros(Int64, length(test[:,1]))))

df_test = DataFrame(NO_OUVRAGE=test[:, :NO_OUVRAGE], DATE=test[:, :DATE], SUMMcTavish=zeros(Int64, length(test[:,1])),SUMBellevue=zeros(Int64, length(test[:,1])),SUMAssomption=zeros(Int64, length(test[:,1])),SUMStHubert=zeros(Int64, length(test[:,1])),SUMPET=zeros(Int64, length(test[:,1])),SUM2McTavish=zeros(Int64, length(test[:,1])),SUM2Bellevue=zeros(Int64, length(test[:,1])),SUM2Assomption=zeros(Int64, length(test[:,1])),SUM2StHubert=zeros(Int64, length(test[:,1])),SUM2PET=zeros(Int64, length(test[:,1])),MAXMcTavish=zeros(Int64, length(test[:,1])),MAXBellevue=zeros(Int64, length(test[:,1])),MAXAssomption=zeros(Int64, length(test[:,1])),MAXStHubert=zeros(Int64, length(test[:,1])),MAXPET=zeros(Int64, length(test[:,1])), SURVERSE=zeros(Float64, length(test[:,1])))
for i=1:length(df_test[:,1])
    row_sum = filter(row-> row.date== df_test[i,2], X_pcp_sum)
    row_sum_2_days = filter(row-> row.date== df_test[i,2], X_pcp_two_days)
    row_max = filter(row-> row.date== df_test[i,2], X_pcp_max)
    for j=2:6
        df_test[i, j+1] = row_sum[1, j]
    end
    for j=7:11
        df_test[i, j+1] = row_sum_2_days[1, j-5]
    end
    for j=12:16
        df_test[i, j+1] = row_max[1, j-10]
    end
end


M_3260 = TrainLogit("3260-01D", surverse_df)
M_3350 = TrainLogit("3350-07D", surverse_df)
M_4240 = TrainLogit("4240-01D", surverse_df)
M_4350 = TrainLogit("4350-01D", surverse_df)
M_4380 = TrainLogit("4380-01D", surverse_df)

df_3260 = filter(row -> row.NO_OUVRAGE == "3260-01D", df_test)
df_3350 = filter(row -> row.NO_OUVRAGE == "3350-07D", df_test)
df_4240 = filter(row -> row.NO_OUVRAGE == "4240-01D", df_test)
df_4350 = filter(row -> row.NO_OUVRAGE == "4350-01D", df_test)
df_4380 = filter(row -> row.NO_OUVRAGE == "4380-01D", df_test)


# Estimation de la probabilité de surverse de l'ensemble de test
X̃ = hcat(ones(Float64,length(df_test[:,1])),df_test[:,:SUMMcTavish], df_test[:,:SUMBellevue], df_test[:,:SUMAssomption], df_test[:,:SUMStHubert], df_test[:,:SUMPET], df_test[:, :SUM2McTavish], df_test[:, :SUM2Bellevue], df_test[:, :SUM2Assomption], df_test[:, :SUM2StHubert], df_test[:, :SUM2PET], df_test[:, :MAXMcTavish], df_test[:, :MAXBellevue], df_test[:, :MAXAssomption], df_test[:, :MAXStHubert], df_test[:, :MAXPET])
X̃_3260 = hcat(ones(Float64,length(df_3260[:,1])),df_3260[:,:SUMMcTavish], df_3260[:,:SUMBellevue], df_3260[:,:SUMAssomption], df_3260[:,:SUMStHubert], df_3260[:,:SUMPET], df_3260[:, :SUM2McTavish], df_3260[:, :SUM2Bellevue], df_3260[:, :SUM2Assomption], df_3260[:, :SUM2StHubert], df_3260[:, :SUM2PET], df_3260[:, :MAXMcTavish], df_3260[:, :MAXBellevue], df_3260[:, :MAXAssomption], df_3260[:, :MAXStHubert], df_3260[:, :MAXPET])
X̃_3350 = hcat(ones(Float64,length(df_3350[:,1])),df_3350[:,:SUMMcTavish], df_3350[:,:SUMBellevue], df_3350[:,:SUMAssomption], df_3350[:,:SUMStHubert], df_3350[:,:SUMPET], df_3350[:, :SUM2McTavish], df_3350[:, :SUM2Bellevue], df_3350[:, :SUM2Assomption], df_3350[:, :SUM2StHubert], df_3350[:, :SUM2PET], df_3350[:, :MAXMcTavish], df_3350[:, :MAXBellevue], df_3350[:, :MAXAssomption], df_3350[:, :MAXStHubert], df_3350[:, :MAXPET])
X̃_4240 = hcat(ones(Float64,length(df_4240[:,1])),df_4240[:,:SUMMcTavish], df_4240[:,:SUMBellevue], df_4240[:,:SUMAssomption], df_4240[:,:SUMStHubert], df_4240[:,:SUMPET], df_4240[:, :SUM2McTavish], df_4240[:, :SUM2Bellevue], df_4240[:, :SUM2Assomption], df_4240[:, :SUM2StHubert], df_4240[:, :SUM2PET], df_4240[:, :MAXMcTavish], df_4240[:, :MAXBellevue], df_4240[:, :MAXAssomption], df_4240[:, :MAXStHubert], df_4240[:, :MAXPET])
X̃_4350 = hcat(ones(Float64,length(df_4350[:,1])),df_4350[:,:SUMMcTavish], df_4350[:,:SUMBellevue], df_4350[:,:SUMAssomption], df_4350[:,:SUMStHubert], df_4350[:,:SUMPET], df_4350[:, :SUM2McTavish], df_4350[:, :SUM2Bellevue], df_4350[:, :SUM2Assomption], df_4350[:, :SUM2StHubert], df_4350[:, :SUM2PET], df_4350[:, :MAXMcTavish], df_4350[:, :MAXBellevue], df_4350[:, :MAXAssomption], df_4350[:, :MAXStHubert], df_4350[:, :MAXPET])
X̃_4380 = hcat(ones(Float64,length(df_4380[:,1])),df_4380[:,:SUMMcTavish], df_4380[:,:SUMBellevue], df_4380[:,:SUMAssomption], df_4380[:,:SUMStHubert], df_4380[:,:SUMPET], df_4380[:, :SUM2McTavish], df_4380[:, :SUM2Bellevue], df_4380[:, :SUM2Assomption], df_4380[:, :SUM2StHubert], df_4380[:, :SUM2PET], df_4380[:, :MAXMcTavish], df_4380[:, :MAXBellevue], df_4380[:, :MAXAssomption], df_4380[:, :MAXStHubert], df_4380[:, :MAXPET])

df_3260.SURVERSE = predict(M_3260, X̃_3260)
df_3350.SURVERSE = predict(M_3350, X̃_3350)
df_4240.SURVERSE = predict(M_4240, X̃_4240)
df_4350.SURVERSE = predict(M_4350, X̃_4350)
df_4380.SURVERSE = predict(M_4380, X̃_4380)

Ỹ = DataFrame(DATE=df_test[:,:DATE], NO_OUVRAGE=df_test[:,:NO_OUVRAGE], SURVERSE=zeros(Int64,length(df_test[:,1])))
for i=1:length(df_test[:,1])
    if df_test[i, :NO_OUVRAGE] == "3260-01D"
        ind = findfirst(df_3260[:, :DATE] .== df_test[i, :DATE])
        df_test[i,:SURVERSE] = df_3260[ind, :SURVERSE]
    elseif df_test[i, :NO_OUVRAGE] == "3350-07D"
        ind = findfirst(df_3350[:, :DATE] .== df_test[i, :DATE])
        df_test[i,:SURVERSE] = df_3350[ind, :SURVERSE]
    elseif df_test[i, :NO_OUVRAGE] == "4240-01D"
        ind = findfirst(df_4240[:, :DATE] .== df_test[i, :DATE])
        df_test[i,:SURVERSE] = df_4240[ind, :SURVERSE]
    elseif df_test[i, :NO_OUVRAGE] == "4350-01D"
        ind = findfirst(df_4350[:, :DATE] .== df_test[i, :DATE])
        df_test[i,:SURVERSE] = df_4350[ind, :SURVERSE]
    elseif df_test[i, :NO_OUVRAGE] == "4380-01D"
        ind = findfirst(df_4380[:, :DATE] .== df_test[i, :DATE])
        df_test[i,:SURVERSE] = df_4380[ind, :SURVERSE]
    end
end
for i=1:length(df_test[:,1])
    if df_test[i,:SURVERSE] > .5
        Ỹ[i, :SURVERSE] = 1
    else
        Ỹ[i, :SURVERSE] = 0
    end
end

# Pour chacune des lignes du fichier test, comportant un ouvrage et une date, une prédiction est requise.
# Dans ce cas-ci, utilisons une prédiction les plus naîve. 
# On prédit avec une chance sur deux qu'il y ait surverse, sans utiliser de variables explicatives
#n = size(test,1)
#surverse = rand(n) .> .5


# Création du fichier sampleSubmission.csv pour soumettre sur Kaggle
ID = test[:,:NO_OUVRAGE].*"_".*string.(test[:,:DATE])
sampleSubmission = DataFrame(ID = ID, Surverse=Ỹ[:,:SURVERSE])
CSV.write("sampleSubmission.csv",sampleSubmission)

# Vous pouvez par la suite déposer le fichier sampleSubmission.csv sur Kaggle.

"sampleSubmission.csv"